In [1]:
%matplotlib qt
# %matplotlib ipympl

In [2]:
## Imports
print("Importing...")
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import cv2
import easyocr

from matplotlib.backends.backend_pdf import PdfPages
import recognizer_modules


Importing...


In [26]:
########################################################
##########               INPUTS               ##########
########################################################

VIDEO_PATH = r"TestVideos\Test_window.avi"
# VIDEO_PATH = None

In [4]:
CAP = cv2.VideoCapture(VIDEO_PATH)

In [14]:
## PreProcessor settings
rules = dict(re_rule=r'-?\d{1,3}\.\d', )
variable_patterns = {'Viscosity': rules, 'Temperature': rules}


class PreProcessor(recognizer_modules.PreProcessor):
    Blur = range(1, 50)

    def process(self, image):
        image = cv2.blur(image, (int(self['Blur']), int(self['Blur'])))
        try:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        except:
            pass
        image = cv2.bitwise_not(image)
        return image
    
if VIDEO_PATH is None:
    input_path = ''
    while input_path == '':
        input_path = input(f"Input video path: ")
    VIDEO_PATH = input_path

CAP = cv2.VideoCapture(VIDEO_PATH)
FPS = int(CAP.get(cv2.CAP_PROP_FPS))
LENTH = int(CAP.get(cv2.CAP_PROP_FRAME_COUNT) / FPS)
CAP.set(cv2.CAP_PROP_POS_FRAMES, 0)
_, START_FRAME = CAP.read()

processor = PreProcessor([i for i in variable_patterns])
processor.configure_process(CAP)
processor.select_window(CAP)
processor.check_process(CAP)
print('Configurating end')

Configurate image processing
Configurating end


d:\WORKS\Diplom-work\recognizer_modules.py:150: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  for variable in self.variable_windows:
d:\WORKS\Diplom-work\recognizer_modules.py:150: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  for variable in self.variable_windows:


In [5]:
import time

In [15]:
CAP.set(cv2.CAP_PROP_POS_FRAMES, 0)
_, START_FRAME = CAP.read()
START_FRAME

In [25]:
import cv2

# CAP = cv2.VideoCapture(VIDEO_PATH)
# CAP.set(cv2.CAP_PROP_POS_FRAMES, 0)
_, START_FRAME = CAP.read()
cap = CAP
cv2.namedWindow('Frame')
# if not cap.isOpened():
#     raise NameError

# Our ROI, defined by two points
p1, p2 = (0, 0), (0, 0)
drawing = False  # True while ROI is actively being drawn by mouse
show_drawing = False  # True while ROI is drawn but is pending use or cancel
blue_color = (255, 0, 0)


def on_mouse(event, x, y, flags, userdata):
    global p1, p2, drawing, show_drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        # Left click down (select first point)
        drawing = True
        show_drawing = True
        p1 = x, y
        p2 = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        # Drag to second point
        if drawing:
            p2 = x, y
    elif event == cv2.EVENT_LBUTTONUP:
        # Left click up (select second point)
        drawing = False
        p2 = x, y


cv2.setMouseCallback('Frame', on_mouse)

while True:
    val, fr = cap.read()
    if not val:
        print('Error with video capterer')
        cv2.destroyAllWindows()
        break

    if show_drawing:
        p2 = (0 if p2[0] < 0 else
              (p2[0] if p2[0] < fr.shape[1] else fr.shape[1]),
              0 if p2[1] < 0 else
              (p2[1] if p2[1] < fr.shape[0] else fr.shape[0]))
        cv2.rectangle(fr, p1, p2, blue_color, 2)
        # avg_y = (p1[1] + p2[1]) // 2
        # cv2.line(fr, (p1[0], avg_y), (p2[0], avg_y), blue, 2)  # Middle horizontal line
        # avg_x = (p1[0] + p2[0]) // 2
        # cv2.line(fr, (avg_x, p1[1]), (avg_x, p2[1]), blue, 2)  # Middle vertical line

    cv2.imshow('Frame', fr)

    keyboard = cv2.waitKey(1)
    if keyboard in [13, 32]:
        # Pressed Enter or Space to use ROI
        drawing = False
        # show_drawing = False
        cv2.destroyAllWindows()
        break
        
    elif keyboard in [ord('c'), ord('C'), 27]:
        CAP.set(cv2.CAP_PROP_POS_FRAMES, 0)
        # Pressed C or Esc to cancel ROI
    elif keyboard in [ord('q'), ord('Q')]:
        # Pressed Q to exit
        cv2.destroyAllWindows()
        break

    # time.sleep(0.01)

# cap.release()
p1, p2


Error with video capterer


((0, 0), (0, 0))

In [ ]:
class PreProcessor:
    _parametr_configurations = []
    parametrs = {}

    def configure_process(
        self,
        video_capture,
        start_frame: int = 0,
        end_frame: int = 0,
    ):

        def update(val):
            time = TIME_slider.val
            video_capture.set(cv2.CAP_PROP_POS_FRAMES, int(fps * time))
            _, image = video_capture.read()

            for slider in sliders:
                slider_name = str(slider.label).split("'")[1]
                self[slider_name] = slider.val

            image_processed = self.process(image)
            plot.set_data(image_processed)
            plot.autoscale()
            fig.canvas.draw_idle()

        fig, ax = plt.subplots()
        fig.set_size_inches(5, 5)
        fig.subplots_adjust(
            left=0.25,
            right=1,
            bottom=0.25,
            top=1,
            hspace=0,
            wspace=0,
        )
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        _, image = video_capture.read()

        plot = ax.imshow(self.process(image), cmap='binary')

        time_slider_ax = fig.add_axes([0.25, 0.1, 0.65, 0.03])
        fps = int(video_capture.get(cv2.CAP_PROP_FPS))
        max_len = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT) / fps) - 1
        end_frame = max_len if not end_frame else end_frame

        TIME_slider = Slider(
            ax=time_slider_ax,
            label='Time',
            valmin=start_frame,
            valmax=end_frame,
            valinit=start_frame,
            valstep=1,
        )
        TIME_slider.on_changed(update)

        sliders = []
        ofset = 0.2
        for parametr, diap in self._parametr_configurations.items():
            slider_ax = fig.add_axes([ofset, 0.25, 0.03, 0.6])

            p_min = min(diap)
            p_max = max(diap)
            p_step = (max(diap) - min(diap)) / (len(diap) - 1)

            slider = Slider(
                ax=slider_ax,
                orientation='vertical',
                label=parametr,
                valmin=p_min,
                valmax=p_max,
                valinit=self[parametr],
                valstep=p_step,
            )
            slider.on_changed(update)
            sliders.append(slider)
            ofset -= 0.02

        print('Configurate image processing')
        plt.show()

    def select_window(self, video_capture, i_frame=0):
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, i_frame)
        _, image = video_capture.read()

        for variable in self.variable_windows:
            processed_image = self.process(image)
            processed_image = cv2.bitwise_not(processed_image)
            # TODO: make border color=red
            window = cv2.selectROI(
                f"Select {variable}",
                processed_image,
                fromCenter=False,
                showCrosshair=True,
            )
            self.variable_windows[variable] = window
        cv2.destroyAllWindows()
    def select_window2(self, video_capture, i_frame=0):
        
        
        ...
    def check_process(
        self,
        video_capture,
        start_frame: int = 0,
        end_frame: int = 0,
    ):

        def update(val):
            time = TIME_slider.val
            video_capture.set(cv2.CAP_PROP_POS_FRAMES, int(fps * time))
            _, image = video_capture.read()

            image_processed = self.process(image)
            stricted_images_list = self.strict(image_processed)

            i = 0
            for variable in self.variable_windows:
                plots[i].set_data(stricted_images_list[variable])
                plots[i].autoscale()
                i += 1

            fig.canvas.draw_idle()

        fig, axises = plt.subplots(nrows=len(self.variable_windows) + 1)
        fig.set_size_inches(5, 5)
        fig.subplots_adjust(
            left=0.1,
            right=0.9,
            bottom=0.0,
            top=1,
            hspace=0.0,
            wspace=0.1,
        )
        if not isinstance(axises, np.ndarray): axises = [axises]

        time_slider_ax = axises[0]
        axises = axises[1:]

        video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        _, image = video_capture.read()
        image_processed = self.process(image)
        stricted_images_list = self.strict(image_processed)

        plots = []
        i = 0
        for variable in self.variable_windows:
            plots.append(
                axises[i].imshow(
                    stricted_images_list[variable],
                    cmap='binary',
                ), )
            i += 1
        fps = int(video_capture.get(cv2.CAP_PROP_FPS))
        max_len = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT) / fps) - 1
        end_frame = max_len if not end_frame else end_frame

        TIME_slider = Slider(
            ax=time_slider_ax,
            label='Time',
            valmin=start_frame,
            valmax=end_frame,
            valinit=start_frame,
            valstep=1,
        )
        TIME_slider.on_changed(update)
        plt.show()

    def strict(self, image: np.ndarray) -> np.ndarray:
        images = {}
        for variable, window in self.variable_windows.items():
            x, y, dx, dy = window
            images[variable] = image[y:y + dy, x:x + dx]
        return images

    def process(self, image: np.ndarray) -> np.ndarray:
        raise NotImplementedError

    def __init__(self, variables):
        all_fields = dict(self.__class__.__dict__)
        self._parametr_configurations = {
            key: value
            for key, value in all_fields.items()
            if key[0].isupper()
        }
        self.parametrs = {
            key: min(value)
            for key,
            value in self._parametr_configurations.items()
        }
        self.variable_windows = {variable: 0 for variable in variables}

    def __getitem__(self, item):
        return self.parametrs[item]

    def __setitem__(self, item, value):
        self.parametrs[item] = value

    def __call__(self, image) -> np.ndarray:
        return self.process(image)



In [28]:
import time

In [27]:
cv2.destroyAllWindows()

In [ ]:
## PostProcessor settings
class PostProcessor(recognizer_modules.PostProcessor):
    def pattern(
        self,
        value: list,
        re_rule=None, min_rule=None, max_rule=None,

    ) -> float|None:

        if value == []: return None
        value = value[0]
        value = value.replace(',', '.')
        regexp_cond = len(re.findall(re_rule, value)) == 1
        try:
            value = float(value)
        except ValueError:
            return None
        min_cond = value <= min_rule if min_rule is not None else True
        max_cond = value >= max_rule if max_rule is not None else True

        return value if regexp_cond and min_cond and max_cond else None
    
    @recognizer_modules.PostProcessor._check_type
    def processor_sweep(self)->list[str]:
        for i in range(1, 50):
            self.inner_processor['Blur'] = i
            processed_img = self.inner_processor(self._image)
            raw_value = [
                value for _, value, _ in self._reader.readtext(processed_img)
            ]

            result = self.pattern(raw_value,**self._rules)
            if result is not None: return raw_value
        return []

    @recognizer_modules.PostProcessor._check_type
    def value_combine(self) -> list[str]:
        parts = len(self._raw_value)
        if parts == 1:
            value = self._raw_value[0]
            result = value[:3] + '.' + value[4:5]

        elif parts == 2:
            result = '.'.join(self._raw_value)

        elif parts == 3:
            result = f'{self._raw_value[0]}.{self._raw_value[2]}'

        return [result]
    
print('Starting recognizer...')
reader = easyocr.Reader(['en'])
checker=PostProcessor(reader=reader, processor=processor)
print([i for i in checker.all_checks])
# checker.active_checks_order = {check:checker.all_checks[check] for check in ['inner_processor_check','value_combine']}

In [ ]:
## Recognize
input_fps = input('Input number of frames per second: ')
try:
    read_fps = float(input_fps)
except:
    read_fps = 1

print('Recognizing:')
errors = 0
frame_line = tqdm(iterable=range(0, FPS * LENTH, int(FPS / read_fps)))
frame_line.set_description(f'Errors: {errors: >4}')
data = []

for i_frame in frame_line:
    CAP.set(cv2.CAP_PROP_POS_FRAMES, i_frame)
    _, frame = CAP.read()
    i_text = {'time': round(i_frame / FPS, 1)}
    processed_frame = processor(frame)
    stricted_images = processor.strict(processed_frame)

    for var, rules in variable_patterns.items():
        var_image = stricted_images[var]
        raw_value = [
            value for _, value, _ in reader.readtext(var_image)
        ]

        mark, result = checker.check(image=var_image,
                               raw_value=raw_value,
                               rules=rules)
        i_text[var] = result
        i_text[var + '_verbose'] = mark

    if None in i_text.values():
        errors += 1
        frame_line.set_description(f'Errors: {errors: >4}')
    data.append(i_text)


In [ ]:
## Print
print(pd.DataFrame(data))